# Visualization: Trading Session

In [1]:
import pandas as pd
import numpy as np

import altair as alt
import seaborn as sns

### 1. Define parameters and Load model

In [2]:
from trading_bot.agent import Agent

model_name = 'model_vic_dqn_50'
test_stock = 'data/VIC2024.csv'
window_size = 10
debug = True

agent = Agent(window_size, pretrained=True, model_name=model_name)

Using TensorFlow backend.


OSError: Unable to open file (file signature not found)

### 2. Load test data

In [10]:
# read csv into dataframe
df = pd.read_csv(test_stock)
# filter out the desired features
df = df[['Date', 'Price']]
# rename feature column names
df = df.rename(columns={'Price': 'actual', 'Date': 'date'})
# convert dates from object to DateTime type
dates = df['date']
dates = pd.to_datetime(dates, infer_datetime_format=True)
df['date'] = dates

df.head()

,date,actual
0,2023-01-03,56800
1,2023-01-04,56000
2,2023-01-05,56200
3,2023-01-06,56100
4,2023-01-09,55000


### 3. Running Eval

In [11]:
import logging
import coloredlogs

from trading_bot.utils import show_eval_result, switch_k_backend_device, get_stock_data
from trading_bot.methods import evaluate_model

# Set logging level to DEBUG
logging.basicConfig(level=logging.DEBUG)

coloredlogs.install(level='DEBUG')
switch_k_backend_device()

test_data = get_stock_data(test_stock)
initial_offset = test_data[1] - test_data[0]
test_result, history = evaluate_model(agent, test_data, window_size, debug)
show_eval_result(model_name, test_result, initial_offset)


2024-03-18 08:50:54 DESKTOP-RSIH8GO root[4808] DEBUG switching to TensorFlow for CPU
2024-03-18 08:50:54 DESKTOP-RSIH8GO root[4808] DEBUG Buy at: 56800.00VND
2024-03-18 08:50:55 DESKTOP-RSIH8GO root[4808] DEBUG Sell at: 56200.00VND | Position: -600.00VND
2024-03-18 08:50:55 DESKTOP-RSIH8GO root[4808] DEBUG Buy at: 54500.00VND
2024-03-18 08:50:55 DESKTOP-RSIH8GO root[4808] DEBUG Sell at: 55000.00VND | Position: +500.00VND
2024-03-18 08:50:55 DESKTOP-RSIH8GO root[4808] DEBUG Buy at: 52700.00VND
2024-03-18 08:50:55 DESKTOP-RSIH8GO root[4808] DEBUG Buy at: 53300.00VND
2024-03-18 08:50:55 DESKTOP-RSIH8GO root[4808] DEBUG Buy at: 52800.00VND
2024-03-18 08:50:55 DESKTOP-RSIH8GO root[4808] DEBUG Buy at: 54500.00VND
2024-03-18 08:50:55 DESKTOP-RSIH8GO root[4808] DEBUG Buy at: 51900.00VND
2024-03-18 08:50:55 DESKTOP-RSIH8GO root[4808] DEBUG Buy at: 52400.00VND
2024-03-18 08:50:55 DESKTOP-RSIH8GO root[4808] DEBUG Buy at: 53500.00VND
2024-03-18 08:50:55 DESKTOP-RSIH8GO root[4808] DEBUG Buy at: 501

In [12]:
history

[(56800, 'BUY'),
 (56000, 'HOLD'),
 (56200, 'SELL'),
 (56100, 'HOLD'),
 (55000, 'HOLD'),
 (54500, 'BUY'),
 (55100, 'HOLD'),
 (55000, 'SELL'),
 (55000, 'HOLD'),
 (54800, 'HOLD'),
 (55200, 'HOLD'),
 (56700, 'HOLD'),
 (57400, 'HOLD'),
 (59200, 'HOLD'),
 (57700, 'HOLD'),
 (57100, 'HOLD'),
 (55200, 'HOLD'),
 (55800, 'HOLD'),
 (56000, 'HOLD'),
 (56000, 'HOLD'),
 (55000, 'HOLD'),
 (54400, 'HOLD'),
 (54200, 'HOLD'),
 (54000, 'HOLD'),
 (53400, 'HOLD'),
 (52900, 'HOLD'),
 (52900, 'HOLD'),
 (53400, 'HOLD'),
 (53500, 'HOLD'),
 (54100, 'HOLD'),
 (54300, 'HOLD'),
 (53000, 'HOLD'),
 (53000, 'HOLD'),
 (52900, 'HOLD'),
 (52500, 'HOLD'),
 (52600, 'HOLD'),
 (52700, 'HOLD'),
 (52800, 'HOLD'),
 (52700, 'HOLD'),
 (52600, 'HOLD'),
 (52700, 'BUY'),
 (53000, 'HOLD'),
 (53000, 'HOLD'),
 (53100, 'HOLD'),
 (53300, 'HOLD'),
 (53000, 'HOLD'),
 (54000, 'HOLD'),
 (53300, 'BUY'),
 (53200, 'HOLD'),
 (52800, 'BUY'),
 (52800, 'HOLD'),
 (52800, 'HOLD'),
 (52800, 'HOLD'),
 (53300, 'HOLD'),
 (53300, 'HOLD'),
 (53400, 'HOLD'

### 4. Visualize

In [15]:
def visualize(df, history, title="trading session"):
    # add history to dataframe
    position = [history[0][0]] + [x[0] for x in history]
    actions = ['HOLD'] + [x[1] for x in history]
    df['position'] = position
    df['action'] = actions
    
    # specify y-axis scale for stock prices
    scale = alt.Scale(domain=(min(min(df['actual']), min(df['position'])) - 50, max(max(df['actual']), max(df['position'])) + 50), clamp=True)
    
    # plot a line chart for stock positions
    actual = alt.Chart(df).mark_line(
        color='green',
        opacity=0.5
    ).encode(
        x='date:T',
        y=alt.Y('position', axis=alt.Axis(format='.2f', title='Price'), scale=scale)
    ).interactive(
        bind_y=False
    )
    
    # plot the BUY and SELL actions as points
    points = alt.Chart(df).transform_filter(
        alt.datum.action != 'HOLD'
    ).mark_point(
        filled=True
    ).encode(
        x=alt.X('date:T', axis=alt.Axis(title='Date')),
        y=alt.Y('position', axis=alt.Axis(format='.2f', title='Price (VND'), scale=scale),
        color='action'
    ).interactive(bind_y=False)

    # merge the two charts
    chart = alt.layer(actual, points, title=title).properties(height=300, width=1000)
    
    return chart

In [16]:
chart = visualize(df, history, title=test_stock)
chart

TypeError: '<' not supported between instances of 'int' and 'str'

In [17]:
df.head()

,date,actual,position,action
0,2023-01-03,56800,56800,HOLD
1,2023-01-04,56000,56800,BUY
2,2023-01-05,56200,56000,HOLD
3,2023-01-06,56100,56200,SELL
4,2023-01-09,55000,56100,HOLD


In [18]:
import pandas as pd
import os

# Đường dẫn đến thư mục
directory = r'D:\Test1\trading-bot-master\extra'

# Đảm bảo rằng đường dẫn tồn tại
if not os.path.exists(directory):
    os.makedirs(directory)

# Lưu DataFrame thành file CSV
df.to_csv(os.path.join(directory, f'{model_name}_result.csv'), index=False)
